https://app.pluralsight.com/guides/building-features-from-text-data/

# Building feature from text data

## Introduction

Text data is different from structured tabular data and, therefore, building features on it requires a completely different approach. In this guide, you will learn how to extract features from raw text for predictive modeling. You will also learn how to perform text preprocessing steps, and create Tf-Idf and Bag-of-words (BOW) feature matrices. We will begin by exploring the data.


## Data

In this guide, we will be using tweet data about the company 'Apple'. The objective is to create features that can be used for building a sentiment predictor model.

The dataset contains 1181 observations and 3 variables, as described below:

- Tweet: Consists of the twitter comments by the users. The twitter data is publicly available.

- Avg: Average sentiment of the tweets (-2 means extremely negative while +2 means extremely positive). This classification was done using the Amazon Mechanical Turk.

- Sentiment: Consists of the sentiment labels - positive, negative, and neutral.

## Loading the Required Libraries and Modules

In [3]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.corpus import stopwords
stop = stopwords.words('english')

## Loading the Data and Performing Basic Data Checks

The first line of code below reads in the data as pandas dataframe, while the second line prints the shape - 1,181 observations of 3 variables.

The third line prints the first five observations.

**WARNING** this dataset was not available to the lesson :( so it is derived from a partial example.

(the following similar dataset was discovered online, but will need processing to work for this notebook)

https://data.world/crowdflower/apple-twitter-sentiment/workspace/file?filename=Apple-Twitter-Sentiment-DFE.csv

In [6]:
dat = pd.read_csv('datatweets.csv')
print(dat.shape)
dat.head(5)

(3016, 3)


,Tweet,Avg,Sentiment
0,iphone 5c is ugly as heck what the freak @appl,-2.0,Negative
1,freak YOU @APPLE,-2.0,Negative
2,freak you @apple,-2.0,Negative
3,@APPLE YOU RUINED MY LIFE,-2.0,Negative
4,@apple I hate apple!!!!!,-2.0,Negative


We will start by performing basic analysis of the data. The line of code below prints the number of tweets, as per the 'Sentiment' label. The output shows that the highest number of tweets are for the negative sentiment, while the lowest are for the positive sentiment.

In [7]:
# Get the number of dates / entries in each month
dat.groupby('Sentiment')['Tweet'].count()

Sentiment
Negative    1098
Neutral     1549
Positive     369
Name: Tweet, dtype: int64

The sentiment score for the tweets is stored in the variable 'Avg', that ranges from -2 (extremely negative) to +2 (extremely positive). We will explore if there is a difference in the average sentiment scores across the 'sentiment' label. The line of code below performs this task and the output shows that the average negative score is -0.74, while the average positive score is 0.57.

In [8]:
dat.groupby('Sentiment')['Avg'].mean()

Sentiment
Negative   -2.0
Neutral     0.0
Positive    2.0
Name: Avg, dtype: float64

# Building Simple Features from Raw Text

Many simple but important features can be extracted from the raw text data, as discussed below.

## Character Length

The hypothesis is that the length of the characters in a tweet varies across the sentiment it carries. The first line of code below creates a new variable 'character_cnt' that takes in the text from the 'Tweet' variable and calculates the count of the characters in the text. The second line performs the 'groupby' operation on the 'Sentiment' label and prints the average character length across the labels.

The output shows that the neutral sentiments have a lower character count on an average, as compared to the positive and the negative tweets. This inference can be useful for separating the neutral tweets from the other types of tweets.

In [10]:
dat['character_cnt'] = dat['Tweet'].str.len()
dat.groupby('Sentiment')['character_cnt'].mean()

Sentiment
Negative    86.949909
Neutral     81.709490
Positive    84.891599
Name: character_cnt, dtype: float64

## Word Count

Just like the character count in a tweet, the word count can also be a useful feature. The first line of code below creates a new variable 'word_counts' that takes in the text from the 'Tweet' variable and calculates the count of the words in the text. The second line performs the 'groupby' operation on the 'Sentiment' label and prints the average word length across the labels.

The output shows that the negative sentiments have the highest average word count, suggesting that the disappointed customers tend to write longer tweets. This inference can be useful for separating the 'sentiment' labels.

In [18]:
dat['word_counts'] = dat['Tweet'].str.split().str.len()
dat.groupby('Sentiment')['word_counts'].mean()

Sentiment
Negative    15.271403
Neutral     13.078115
Positive    13.945799
Name: word_counts, dtype: float64

## Average Character Length per Word

Since we have created the 'character_cnt' and the 'word_counts' features, it is easy to create the ratio of these two variables that will give the average length of the character per word in each tweet.

The first line of code below creates a new variable 'characters_per_word' that is the ratio of the number of characters and the number of words in a tweet. The second line performs the 'groupby' operation on the 'Sentiment' label and prints the average character length per word across the labels.

The output shows that neutral sentiments have the highest average character length per word. This inference can be useful for separating the 'Sentiment' labels.

In [12]:
dat['characters_per_word'] = dat['character_cnt']/dat['word_counts']
dat.groupby('Sentiment')['characters_per_word'].mean()

Sentiment
Negative    5.792099
Neutral     6.390818
Positive    6.227200
Name: characters_per_word, dtype: float64

## Special Character Count

It is also possible to create a feature that contains the count of special characters like '@' or '#'. The first line of code below creates a new feature 'spl' that takes in the text from the 'Tweet' variable and calculates the count of the words starting with the special character '@'. We use the starts with function for performing this operation. The second line prints the first five observations containing the 'Tweet' and the 'spl' variable.

In [13]:
dat['spl'] = dat['Tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('@')]))
dat[['Tweet','spl']].head()

,Tweet,spl
0,iphone 5c is ugly as heck what the freak @appl,1
1,freak YOU @APPLE,1
2,freak you @apple,1
3,@APPLE YOU RUINED MY LIFE,1
4,@apple I hate apple!!!!!,1


## Number Count

Just like we created a feature on the count of words in a tweet, we can also create a feature on the count of numbers in a tweet. The first line of code below creates a new variable 'num' that takes in the text from the 'Tweet' variable and calculates the count of the numbers in the text. The second line performs the 'groupby' operation on the 'Sentiment' label and prints the average count of numbers across the labels.

The output shows that the neutral sentiment labels have the lowest average count of numbers in a tweet, whereas the negative tweets have the highest average.

In [14]:
#Number of numerics
dat['num'] = dat['Tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
dat.groupby('Sentiment')['num'].mean()

Sentiment
Negative    0.151184
Neutral     0.213686
Positive    0.192412
Name: num, dtype: float64

---

# Pre-processing the Raw Text

So far, we have created simple features from the raw text. We can also create more advanced features but, before that, we will have to clean the text. The common pre-processing steps are summarized below:

1. Removing punctuation - the rule of thumb is to remove everything that is not in the form x,y,z. The first line of code below performs this task.

1. Removing stopwords - these are unhelpful words like 'the', 'is', 'at'. These are not helpful because the frequency of such stopwords is high in the corpus, but they don't help in differentiating the target classes. The removal of Stopwords also reduces the data size. The second line of code below performs this task.

1. Conversion to lowercase - words like 'Phone' and 'phone' need to be considered as one word. Hence, these are converted to lowercase. The third line of code below performs this task.

1. Stemming - the goal of stemming is to reduce the number of inflectional forms of words appearing in the text. This causes words such as “argue”, "argued", "arguing", "argues" to be reduced to their common stem “argu”. There are many ways to perform Stemming, the popular one being the “Porter Stemmer” method by Martin Porter. The fourth to sixth lines of code below perform this task.

The last line of code prints a summary of all the new features that we have built so far.

In [15]:
dat['processedtext'] = dat['Tweet'].str.replace('[^\w\s]','') 
dat['processedtext'] = dat['processedtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
dat['processedtext'] = dat['processedtext'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#Lines 4 to 6
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
dat['processedtext'] = dat['processedtext'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

dat[['character_cnt','word_counts','characters_per_word', 'spl', 'num', 'processedtext']].head()

<ipython-input-15-33935a427b35>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dat['processedtext'] = dat['Tweet'].str.replace('[^\w\s]','')


,character_cnt,word_counts,characters_per_word,spl,num,processedtext
0,46,10,4.600000,1,0,iphon 5c ugli heck freak appl
1,16,3,5.333333,1,0,freak you appl
2,16,3,5.333333,1,0,freak appl
3,25,5,5.000000,1,0,appl you ruin my life
4,24,4,6.000000,1,0,appl i hate appl


---

# Term Frequency-Inverse Document Frequency (TF-IDF) Vector

We have cleaned the text which is now stored in a new variable 'processedtext'. However, in order to use it for building machine learning models, we will have to convert it to word frequency vectors.

One of the most popular methods to do this is through the TF-IDF representation, which is used as a weighting factor in text mining applications. In simple terms, TF-IDF attempts to highlight important words which appear frequently in a document but not across documents. The terms are briefly explained below:

1. Term Frequency (TF): This summarizes the normalized Term Frequency within a document.

1. Inverse Document Frequency (IDF): This reduces the weight of terms that appear a lot across documents.

Now, we will work on creating the TF-IDF vectors for our tweets. The first line of code below imports the 'TfidfVectorizer' from sklearn.feature_extraction.text module. The second line initializes the TfidfVectorizer object, called 'tfidf', while the third line fits and transforms the variable 'processedtext' from the data.

The important arguments we have used in initiating the TfidfVectorizer object are the 'max_features' and the 'ngram_range'. While the 'max_features' argument specifies the maximum number of features to be created, the argument 'ngram_range=(1,1)' specifies that unigrams will be considered for feature creation.

The fourth line prints a summary of the object, which is a sparse matrix containing the number of observations (1181) and the number of features (500).


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=500, lowercase=True, analyzer='word', stop_words= 'english',ngram_range=(1,1))

dat_tfIdf = tfidf.fit_transform(dat['processedtext'])
print(dat_tfIdf)

  (0, 38)	0.33367517418709947
  (0, 218)	0.9426881128619417
  (1, 38)	1.0
  (2, 38)	1.0
  (3, 248)	0.9811380155172429
  (3, 38)	0.193308547423249
  (4, 189)	0.9282015014964591
  (4, 38)	0.3720779120288103
  (5, 438)	0.37834361403311906
  (5, 484)	0.4200934898559464
  (5, 188)	0.42352816849143876
  (5, 432)	0.45948873321342304
  (5, 433)	0.3061266986594431
  (5, 466)	0.3765150190752095
  (5, 38)	0.07782178465306747
  (5, 218)	0.21985954302076122
  (6, 119)	0.3528775517743811
  (6, 278)	0.42530636574548564
  (6, 39)	0.48869488180537557
  (6, 273)	0.47784768113907605
  (6, 466)	0.40541886716797154
  (6, 38)	0.08379591298251557
  (6, 218)	0.23673745362529616
  (7, 381)	0.5838288184018563
  (7, 363)	0.37800913047565576
  :	:
  (3011, 82)	0.5873363377028471
  (3011, 262)	0.2470472603379669
  (3011, 38)	0.06439096954426193
  (3011, 218)	0.18191524650555752
  (3012, 246)	0.45192301735774204
  (3012, 383)	0.38897452405245014
  (3012, 129)	0.45953297622307376
  (3012, 163)	0.3824672494398225
  (

# Bag-of-words Vector

Another popular technique for creating word vectors is the Bag-of-words approach. It is a simplistic method for identifying topics in a document. It works on the assumption that the higher the frequency of the term, the higher its importance.

The first line of code below imports the 'CountVectorizer' utility from the 'sklearn.feature_extraction.text' module. The second line initializes the CountVectorizer object, called 'bag_words', while the third line fits and transforms the variable 'processedtext' from the data. The fourth line prints a summary of the object, which is, again, a sparse matrix containing the number of observations (1181) and the number of features (500).

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
bag_words = CountVectorizer(max_features=500, lowercase=True, ngram_range=(1,1),analyzer = "word")
dat_BOW = bag_words.fit_transform(dat['processedtext'])
dat_BOW

<3016x500 sparse matrix of type '<class 'numpy.int64'>'
	with 18217 stored elements in Compressed Sparse Row format>

# Conclusion

In this guide, you have learned the fundamentals of building features from the raw and the processed text data. You can now use the basic as well as advanced features for building a machine learning algorithm that can predict the sentiment of a tweet.

- [Natural Language Processing - Machine Learning with Text Data](https://app.pluralsight.com/guides/nlp-machine-learning-text-data/)
- [Natural Language Processing – Text Parsing](https://app.pluralsight.com/guides/text-parsing/)
- [Natural Language Processing - Topic Identification](https://app.pluralsight.com/guides/topic-identification-nlp/)
- [Visualizing Text Data Using a Word Cloud](https://app.pluralsight.com/guides/natural-language-processing-visualizing-text-data-using-word-cloud/)
- [Named Entity Recognition (NER)](https://app.pluralsight.com/guides/natural-language-processing-named-entity-recognition/)
- [Natural Language Processing - Extracting Sentiment from the Text Data](https://app.pluralsight.com/guides/natural-language-processing-extracting-sentiment-from-text-data/)